In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tkinter as tk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from wordcloud import WordCloud
from nltk.sentiment import SentimentIntensityAnalyzer

In [2]:
# Load the dataset
df = pd.read_csv("SampleVisualizationDataset.csv")

## GUI for Data Visualization

In [3]:
# Create the GUI window
root = tk.Tk()
root.title("Mental Health Dataset Visualization")

# Create a figure and canvas for the plot
fig = Figure(figsize=(8, 6), dpi=100)
ax = fig.add_subplot(111)
canvas = FigureCanvasTkAgg(fig, master=root)
canvas.draw()
canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

# Define functions for each visualization
def plot_status_correlations():
    ax.clear()    
    weekday_status_matrix = df.pivot_table(index="Text", columns="Status", aggfunc="size", fill_value=0)
    corr_matrix = weekday_status_matrix.corr()
    sns.heatmap(corr_matrix, ax=ax, annot=True, cmap='coolwarm',cbar=False)
    ax.set_title("Status Correlations")
    canvas.draw()
 
def plot_day_of_week_distribution():
    ax.clear()
    df['Weekday'].value_counts().plot(kind='bar', ax=ax)
    ax.set_title("Day of Week Distribution")
    canvas.draw()

def plot_status_over_time():
    ax.clear()
    df.groupby('Weekday')['Status'].value_counts().unstack().plot(kind='line', ax=ax)
    ax.set_title("Status Over Time")
    canvas.draw()

def plot_word_cloud():
    ax.clear()
    wordcloud = WordCloud().generate(' '.join(df['Text']))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis("off")
    ax.set_title("Word Cloud")
    canvas.draw()

def plot_pie_chart():
    ax.clear()
    df['Status'].value_counts().plot(kind='pie', ax=ax, autopct='%1.1f%%')
    ax.set_title("Status Pie Chart")
    canvas.draw()

def plot_status_correlation():
    ax.clear()
    corr_matrix = pd.crosstab(df["Status"], df["Status"])
    sns.heatmap(corr_matrix, ax=ax, annot=True, cmap="coolwarm", square=True ,cbar=False )
    ax.set_title("Correlation Heatmap")
    canvas.draw()

def plot_line_graph():
    ax.clear()
    weekday_status_counts = df.groupby(["Weekday", "Status"]).size().unstack()
    weekday_status_counts.plot(kind="bar", ax=ax)
    ax.set_title("Status Distribution by Weekday")
    ax.set_xlabel("Weekday")
    ax.set_ylabel("Count")
    ax.legend(title="Status")
    canvas.draw()

def plot_scoreboard():
    ax.clear()
    status_counts = df["Status"].value_counts()
    scoreboard = pd.DataFrame({
        "Status": status_counts.index,
        "Count": status_counts.values,
        "Percentage": (status_counts.values / len(df)) * 100
    })
    scoreboard["Rank"] = scoreboard["Count"].rank(ascending=False).astype(int)
    ax.axis('off')
    ax.table(cellText=scoreboard.values, colLabels=scoreboard.columns, loc="center")
    canvas.draw()


def plot_sentiment_analysis():
    ax.clear()
    sia = SentimentIntensityAnalyzer()
    sentiments = df["Text"].apply(lambda x: sia.polarity_scores(x)["compound"])
    ax.bar(range(len(sentiments)), sentiments)
    ax.set_xlabel("Text Index")
    ax.set_ylabel("Sentiment Score")
    ax.set_title("Sentiment Analysis")
    canvas.draw()

# Create buttons for each visualization
button_frame = tk.Frame(root)
button_frame.pack(side=tk.BOTTOM)

button1 = tk.Button(button_frame, text="Status Distribution", command=plot_status_correlations)
button1.pack(side=tk.LEFT)

button2 = tk.Button(button_frame, text="Day of Week Distribution", command=plot_day_of_week_distribution)
button2.pack(side=tk.LEFT)

button3 = tk.Button(button_frame, text="Status Over Time", command=plot_status_over_time)
button3.pack(side=tk.LEFT)

button4 = tk.Button(button_frame, text="Word Cloud", command=plot_word_cloud)
button4.pack(side=tk.LEFT)

button5 = tk.Button(button_frame, text="Pie Chart", command=plot_pie_chart)
button5.pack(side=tk.LEFT)

button6 = tk.Button(button_frame, text="Status Correlation", command=plot_status_correlation)
button6.pack(side=tk.LEFT)

button7 = tk.Button(button_frame, text="Line Graph", command=plot_line_graph)
button7.pack(side=tk.LEFT)

button8 = tk.Button(button_frame, text="Scoreboard", command=plot_scoreboard)
button8.pack(side=tk.LEFT)

button9 = tk.Button(button_frame, text="Sentiment Analysis", command=plot_sentiment_analysis)
button9.pack(side=tk.LEFT)

# Start the GUI event loop
root.mainloop()

In [4]:
import speech_recognition as sr
import pickle
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import datetime
import joblib as j
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tkinter as tk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from wordcloud import WordCloud
from nltk.sentiment import SentimentIntensityAnalyzer

In [5]:
vector=j.load('vectorizer.pkl')
model=j.load('model.pkl')

## GUI for Speech to text

In [7]:
import tkinter as tk
from tkinter import messagebox
import speech_recognition as sr
import pandas as pd
import os

# Assuming you have 'vector' and 'model' defined elsewhere
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)

def predict_mood(text):
    vec = vector.transform([text])
    result = model.predict(vec)[0]
    return result

def save_to_csv(data, filename='patient_data.csv'):
    df = pd.DataFrame(data)
    df.to_csv(filename, mode='a', header=not os.path.exists(filename), index=False)

class PatientDataCollector:
    def __init__(self, root):
        self.root = root
        self.root.title("Patient Data Collector")
        self.day_label = tk.Label(root, text="Day:")
        self.day_label.pack()
        self.day_entry = tk.Entry(root)
        self.day_entry.pack()
        self.record_button = tk.Button(root, text="Record Audio", command=self.record_audio)
        self.record_button.pack()
        self.text_label = tk.Label(root, text="Text:")
        self.text_label.pack()
        self.text_box = tk.Text(root, height=10, width=40)
        self.text_box.pack()
        self.predict_button = tk.Button(root, text="Predict Mood", command=self.predict_mood)
        self.predict_button.pack()
        self.result_label = tk.Label(root, text="Predicted Mood:")
        self.result_label.pack()
        self.result_box = tk.Text(root, height=1, width=40)
        self.result_box.pack()
        self.save_button = tk.Button(root, text="Save to CSV", command=self.save_to_csv)
        self.save_button.pack()
        

    def record_audio(self):
        r = sr.Recognizer()
        with sr.Microphone() as source:
            print("Please say something:")
            audio = r.listen(source)
            try:
                text = r.recognize_google(audio, language="en-US")
                self.text_box.delete(1.0, tk.END)
                self.text_box.insert(tk.END, text)
            except sr.UnknownValueError:
                messagebox.showerror("Error", "Google Speech Recognition could not understand your audio")
            except sr.RequestError as e:
                messagebox.showerror("Error", "Could not request results from Google Speech Recognition service; {0}".format(e))

    def predict_mood(self):
        text = self.text_box.get(1.0, tk.END)
        preprocessed_text = preprocess_text(text)
        predicted_mood = predict_mood(preprocessed_text)
        self.result_box.delete(1.0, tk.END)
        self.result_box.insert(tk.END, predicted_mood)

    def save_to_csv(self):
        day = self.day_entry.get()
        text = self.text_box.get(1.0, tk.END)
        predicted_mood = self.result_box.get(1.0, tk.END)
        data = [{'day': day, 'text': text, 'predicted_mood': predicted_mood}]
        save_to_csv(data)
        messagebox.showinfo("Success", "Data saved to CSV file")

root = tk.Tk()
collector = PatientDataCollector(root)
root.mainloop()

Please say something:


## GUI for Recommendation

In [17]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt

class MentalHealthAssessmentGUI:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Mental Health Assessment")

        # Create a file selection button
        self.file_button = tk.Button(self.root, text="Select CSV File", command=self.select_file)
        self.file_button.pack()

        # Create a label to display the file name
        self.file_label = tk.Label(self.root, text="")
        self.file_label.pack()

        # Create a button to visualize and assess
        self.assess_button = tk.Button(self.root, text="Visualize and Assess", command=self.visualize_and_assess)
        self.assess_button.pack()

        # Create a text box to display the results
        self.results_text = tk.Text(self.root, width=40, height=10)
        self.results_text.pack()

    def select_file(self):
        filename = filedialog.askopenfilename(title="Select CSV File", filetypes=[("CSV Files", "*.csv")])
        self.file_label.config(text=f"Selected file: {filename}")

    def visualize_and_assess(self):
        filename = self.file_label.cget("text").split(": ")[1]
        df = pd.read_csv(filename)

        mood_counts = df['predicted_mood'].value_counts()

        plt.figure(figsize=(4, 6))
        plt.pie(mood_counts, labels=mood_counts.index, autopct='%1.1f%%', startangle=140)
        plt.title('Mood Distribution')
        plt.show()

        most_common_mood = mood_counts.idxmax()

        health_status = ""
        recommendations = []

        if most_common_mood == 'happy':
            health_status = 'The patient is fit.'
            recommendations = [
                'Continue engaging in activities that bring you joy.',
                'Maintain a healthy lifestyle with balanced nutrition and regular exercise.',
                'Consider setting new personal goals and challenges to stay motivated.'
            ]
        elif most_common_mood == 'Normal':
            health_status = 'The patient is in a normal mental state.'
            recommendations = [
                'Keep up with your current routines and practices.',
                'Stay connected with friends and family to maintain social support.',
                'Regularly check in with yourself to ensure you’re managing stress effectively.'
            ]
        elif most_common_mood == 'depression':
            health_status = 'The patient is depressed.'
            recommendations = [
                'Consider seeking support from a mental health professional.',
                'Engage in activities that you used to enjoy, even if it’s difficult.',
                'Reach out to trusted friends or family members for support and understanding.'
            ]
        elif most_common_mood == 'suicidal':
            health_status = 'The patient is not in a healthy mental state.'
            recommendations = [
                'Seek immediate help from a mental health professional or counselor.',
                'Contact crisis support services or helplines for urgent assistance.',
                'Ensure you have a support network in place and communicate openly about your feelings.'
            ]
        else:
            health_status = 'Unknown mood state.'
            recommendations = [
                'Consider reviewing the mood prediction model for accuracy.',
                'Ensure proper data collection and preprocessing to improve predictions.',
                'Consult with a mental health professional to understand the implications of your data.'
            ]

        results = f"Most common mood: {most_common_mood}\n{health_status}\nRecommendations:\n"
        for suggestion in recommendations:
            results += f"- {suggestion}\n"

        self.results_text.delete(1.0, tk.END)
        self.results_text.insert(tk.END, results)

    def run(self):
        self.root.mainloop()

if __name__ == "__main__":
    gui = MentalHealthAssessmentGUI()
    gui.run()

In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tkinter as tk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from wordcloud import WordCloud
from nltk.sentiment import SentimentIntensityAnalyzer

class MentalHealthDatasetVisualization:
    def __init__(self, root):
        self.root = root
        self.root.title("Mental Health Dataset Visualization")
        self.df = pd.read_csv("SampleVisualizationDataset.csv")
        self.fig = Figure(figsize=(8, 6), dpi=100)
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.fig, master=root)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        self.button_frame = tk.Frame(root)
        self.button_frame.pack(side=tk.BOTTOM)
        self.create_buttons()

    def create_buttons(self):
        button1 = tk.Button(self.button_frame, text="Status Distribution", command=self.plot_status_correlations)
        button1.pack(side=tk.LEFT)

        button2 = tk.Button(self.button_frame, text="Day of Week Distribution", command=self.plot_day_of_week_distribution)
        button2.pack(side=tk.LEFT)

        button3 = tk.Button(self.button_frame, text="Status Over Time", command=self.plot_status_over_time)
        button3.pack(side=tk.LEFT)

        button4 = tk.Button(self.button_frame, text="Word Cloud", command=self.plot_word_cloud)
        button4.pack(side=tk.LEFT)

        button5 = tk.Button(self.button_frame, text="Pie Chart", command=self.plot_pie_chart)
        button5.pack(side=tk.LEFT)

        button6 = tk.Button(self.button_frame, text="Status Correlation", command=self.plot_status_correlation)
        button6.pack(side=tk.LEFT)

        button7 = tk.Button(self.button_frame, text="Line Graph", command=self.plot_line_graph)
        button7.pack(side=tk.LEFT)

        button8 = tk.Button(self.button_frame, text="Scoreboard", command=self.plot_scoreboard)
        button8.pack(side=tk.LEFT)

        button9 = tk.Button(self.button_frame, text="Sentiment Analysis", command=self.plot_sentiment_analysis)
        button9.pack(side=tk.LEFT)

    def plot_status_correlations(self):
        self.ax.clear()
        weekday_status_matrix = self.df.pivot_table(index="Text", columns="Status", aggfunc="size", fill_value=0)
        corr_matrix = weekday_status_matrix.corr()
        sns.heatmap(corr_matrix, ax=self.ax, annot=True, cmap='coolwarm', cbar=False)
        self.ax.set_title("Status Correlations")
        self.canvas.draw()

    def plot_day_of_week_distribution(self):
        self.ax.clear()
        self.df['Weekday'].value_counts().plot(kind='bar', ax=self.ax)
        self.ax.set_title("Day of Week Distribution")
        self.canvas.draw()

    def plot_status_over_time(self):
        self.ax.clear()
        self.df.groupby('Weekday')['Status'].value_counts().unstack().plot(kind='line', ax=self.ax)
        self.ax.set_title("Status Over Time")
        self.canvas.draw()

    def plot_word_cloud(self):
        self.ax.clear()
        wordcloud = WordCloud().generate(' '.join(self.df['Text']))
        self.ax.imshow(wordcloud, interpolation='bilinear')
        self.ax.axis("off")
        self.ax.set_title("Word Cloud")
        self.canvas.draw()

    def plot_pie_chart(self):
        self.ax.clear()
        self.df['Status'].value_counts().plot(kind='pie', ax=self.ax, autopct='%1.1f%%')
        self.ax.set_title("Status Pie Chart")
        self.canvas.draw()

    def plot_status_correlation(self):
        self.ax.clear()
        corr_matrix = pd.crosstab(self.df["Status"], self.df["Status"])
        sns.heatmap(corr_matrix, ax=self.ax, annot=True, cmap="coolwarm", square=True, cbar=False)
        self.ax.set_title("Correlation Heatmap")
        self.canvas.draw()

    def plot_line_graph(self):
        self.ax.clear()
        weekday_status_counts = self.df.groupby(["Weekday", "Status"]).size().unstack()
        weekday_status_counts.plot(kind="bar", ax=self.ax)
        self.ax.set_title("Status Distribution by Weekday")
        self.ax.set_xlabel("Weekday")
        self.ax.set_ylabel("Count")
        self.ax.legend(title="Status")
        self.canvas.draw()

    def plot_scoreboard(self):
        self.ax.clear()
        status_counts = self.df["Status"].value_counts()
        scoreboard = pd.DataFrame({
            "Status": status_counts.index,
            "Count": status_counts.values,
            "Percentage": (status_counts.values / len(self.df)) * 100
        })
        scoreboard["Rank"] = scoreboard["Count"].rank(ascending=False).astype(int)
        self.ax.axis('off')
        self.ax.table(cellText=scoreboard.values, colLabels=scoreboard.columns, loc="center")
        self.canvas.draw()

    def plot_sentiment_analysis(self):
        self.ax.clear()
        sia = SentimentIntensityAnalyzer()
        sentiments = self.df["Text"].apply(lambda x: sia.polarity_scores(x)["compound"])
        self.ax.bar(range(len(sentiments)), sentiments)
        self.ax.set_xlabel("Text Index")
        self.ax.set_ylabel("Sentiment Score")
        self.ax.set_title("Sentiment Analysis")
        self.canvas.draw()

#if __name__ == "__main__":
root = tk.Tk()
gui = MentalHealthDatasetVisualization(root)
root.mainloop()